In [1]:
import loading_from_pretrained as loading
import torch as t

/share/u/caden/.conda/envs/interp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint_index = None
checkpoint_value = None
fold_ln = False
device = "cuda:0"
n_devices = 1
dtype = t.float32
default_prepend_bos = True

official_model_name = "EleutherAI/pythia-1B"

cfg = loading.get_pretrained_model_config(
        official_model_name,
        checkpoint_index=checkpoint_index,
        checkpoint_value=checkpoint_value,
        fold_ln=fold_ln,
        device=device,
        n_devices=n_devices,
        default_prepend_bos=default_prepend_bos,
        dtype=dtype,
        # **from_pretrained_kwargs,
    )

In [3]:
hf_model = None

state_dict = loading.get_pretrained_state_dict(
    official_model_name, cfg, hf_model, dtype=dtype, 
    # **from_pretrained_kwargs
)

In [4]:
import nnsight

nnmodel = nnsight.LanguageModel("EleutherAI/pythia-1B", device_map="cuda:0")
tokenizer = nnmodel.tokenizer

In [5]:
import UnifiedTransformer

default_padding_side = "right"

model = UnifiedTransformer.UnifiedTransformer(
    cfg,
    tokenizer,
    move_to_device=True,
    default_padding_side=default_padding_side,
)

In [6]:
center_writing_weights = True
center_unembed = True
fold_value_biases = True
refactor_factored_attn_matrices = False

model.load_and_process_state_dict(
    state_dict,
    fold_ln=False,
    center_writing_weights=center_writing_weights,
    center_unembed=center_unembed,
    fold_value_biases=fold_value_biases,
    refactor_factored_attn_matrices=refactor_factored_attn_matrices,
)

In [7]:
model.device = t.device("cuda:0")

In [8]:
model

UnifiedTransformer(
  (embed): Embed()
  (blocks): ModuleList(
    (0-15): 16 x TransformerBlock(
      (ln1): LayerNorm(
        (scale_factor): Nneuron(param_shape=None)
        (normalized): Nneuron(param_shape=None)
      )
      (ln2): LayerNorm(
        (scale_factor): Nneuron(param_shape=None)
        (normalized): Nneuron(param_shape=None)
      )
      (attn): Attention(
        (k): Nneuron(param_shape=None)
        (q): Nneuron(param_shape=None)
        (v): Nneuron(param_shape=None)
        (z): Nneuron(param_shape=None)
        (attn_scores): Nneuron(param_shape=None)
        (pattern): Nneuron(param_shape=None)
        (result): Nneuron(param_shape=None)
        (rot_k): Nneuron(param_shape=None)
        (rot_q): Nneuron(param_shape=None)
      )
      (mlp): MLP()
    )
  )
  (ln_final): LayerNorm(
    (scale_factor): Nneuron(param_shape=None)
    (normalized): Nneuron(param_shape=None)
  )
  (unembed): Unembed()
)

In [13]:
nnsight_model = nnsight.LanguageModel(model, tokenizer=tokenizer, device_map="cuda:0")


with nnsight_model.invoke("The Space Needle is in the city of") as invoker:
    pass

print(invoker.output.softmax(-1).argmax(-1))
for t in invoker.output.softmax(-1).argmax(-1):
    out = tokenizer.decode(t)
    print(out)

Moving model to device:  meta
tensor([[  806, 17842,   282,   310,   247,   253,  1232,  4055,  5003]],
       device='cuda:0')
 first Betweenle is a the process center San


In [12]:
with nnmodel.invoke("The Space Needle is in the city of") as invoker:
    pass

print(invoker.output[0].softmax(-1).argmax(-1))
for t in invoker.output[0].softmax(-1).argmax(-1):
    out = tokenizer.decode(t)
    print(out)

tensor([[  806, 17842,   282,   310,   247,   253,  1232,  4055,  5003]],
       device='cuda:0')
 first Betweenle is a the process center San
